In [ ]:
! 
# pip install seqeval
# pip install evaluate
# pip install pandas
# pip install datasets
# pip install torch
# pip install -U transformers>=4.48.0
# pip install flash-attn

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ----------- ---------------------------- 3.1/11.1 MB 15.3 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.1 MB 16.1 MB/s eta 0:00:01
   ------------------------------------- -- 10.5/11.1 MB 17.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\magnu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import json
import torch
import evaluate
import seqeval
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

# Flash Attention for faster training
# from flash_attn import flash_attn_func

torch.backends.cuda.matmul.allow_tf32 = True  # Enable TF32 for better performance
torch.backends.cudnn.benchmark = True  # Enable CuDNN auto-tuner

In [17]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load dataset
DATA_PATH = "tokenized_ner_data_4.json"
with open(DATA_PATH, "r") as f:
    data = json.load(f)

In [ ]:
def preprocess_data(data):
    tokenized_inputs = []
    tokenized_labels = []

    for entry in data:
        tokens = entry["tokens"]
        labels = entry["labels"]

        tokenized_inputs.append(tokens)
        tokenized_labels.append(labels)
    return {"tokens": tokenized_inputs, "labels": tokenized_labels}

dataset = Dataset.from_dict(preprocess_data(data))

In [ ]:
# Load tokenizer and model
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(set([label for entry in data for label in entry["labels"]]))).to(device)